In [1]:
from transformers import ViTConfig, BertConfig, VisionTextDualEncoderConfig, VisionTextDualEncoderModel
from transformers import AutoProcessor, AutoModelForCausalLM, AutoTokenizer, AutoModel
import requests
from PIL import Image
import warnings
import torch
import sys
# from src.dataset import *
import torch.nn as nn

# sys.path.append("")
warnings.filterwarnings('ignore')

cuda_id = 1

/home/ubuntu/.conda/envs/vqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## BLIP 

In [5]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipModel, BlipForQuestionAnswering
image_dir = "/home/ubuntu/vqa/data/train-images"
text_json_path = "/home/ubuntu/vqa/data/evjvqa_train.json"

# dataset = EVJQA(image_dir, text_json_path)
# loader = data.DataLoader(dataset,
#                          batch_size=32,
#                          shuffle=True)


processor = AutoProcessor.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--Salesforce--blip-vqa-capfilt-large/snapshots/c6af15bed424cf343aab3ff3bb31417ba272923a")
tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--Salesforce--blip-vqa-capfilt-large/snapshots/c6af15bed424cf343aab3ff3bb31417ba272923a")
vqa_model = BlipForQuestionAnswering.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--Salesforce--blip-vqa-capfilt-large/snapshots/c6af15bed424cf343aab3ff3bb31417ba272923a").cuda(cuda_id)
blip_model = BlipModel.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--Salesforce--blip-vqa-capfilt-large/snapshots/c6af15bed424cf343aab3ff3bb31417ba272923a").cuda(cuda_id)

Some weights of the model checkpoint at /home/ubuntu/vqa/huggingface_modules/models--Salesforce--blip-vqa-capfilt-large/snapshots/c6af15bed424cf343aab3ff3bb31417ba272923a were not used when initializing BlipModel: ['text_decoder.bert.encoder.layer.2.attention.self.value.bias', 'text_encoder.encoder.layer.1.attention.output.LayerNorm.bias', 'text_encoder.encoder.layer.2.output.dense.bias', 'text_decoder.bert.encoder.layer.7.intermediate.dense.bias', 'text_decoder.bert.encoder.layer.10.output.LayerNorm.bias', 'text_decoder.bert.encoder.layer.5.attention.self.query.bias', 'text_decoder.bert.encoder.layer.0.attention.output.dense.weight', 'text_encoder.encoder.layer.2.attention.self.value.weight', 'text_encoder.encoder.layer.4.attention.self.query.weight', 'text_encoder.encoder.layer.6.crossattention.self.key.bias', 'text_decoder.bert.encoder.layer.11.attention.output.LayerNorm.weight', 'text_encoder.encoder.layer.9.crossattention.self.key.bias', 'text_decoder.bert.encoder.layer.8.output.L

In [41]:
for name, module in blip_model.named_modules():
    print(name)


text_model
text_model.embeddings
text_model.embeddings.word_embeddings
text_model.embeddings.position_embeddings
text_model.embeddings.LayerNorm
text_model.embeddings.dropout
text_model.encoder
text_model.encoder.layer
text_model.encoder.layer.0
text_model.encoder.layer.0.attention
text_model.encoder.layer.0.attention.self
text_model.encoder.layer.0.attention.self.query
text_model.encoder.layer.0.attention.self.key
text_model.encoder.layer.0.attention.self.value
text_model.encoder.layer.0.attention.self.dropout
text_model.encoder.layer.0.attention.output
text_model.encoder.layer.0.attention.output.dense
text_model.encoder.layer.0.attention.output.LayerNorm
text_model.encoder.layer.0.attention.output.dropout
text_model.encoder.layer.0.crossattention
text_model.encoder.layer.0.crossattention.self
text_model.encoder.layer.0.crossattention.self.query
text_model.encoder.layer.0.crossattention.self.key
text_model.encoder.layer.0.crossattention.self.value
text_model.encoder.layer.0.crossatte

In [5]:
t5_tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d")
t5_model = AutoModel.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d")

Some weights of the model checkpoint at /home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
t5_model.shared

Embedding(250112, 512)

## TEST MODEL 

In [6]:
inputs = dict([(key, value.cuda(cuda_id)) for key, value in next(iter(loader)).items()])
generate_inputs = inputs.pop("labels")

NameError: name 'loader' is not defined

In [10]:
inputs.keys()

dict_keys(['pixel_values', 'input_ids', 'attention_mask'])

In [12]:
generate_outputs = vqa_model.generate(**inputs)

In [14]:
processor.decode(generate_outputs[0])

'yellow [SEP] [PAD] [PAD]'

In [5]:
logits = outputs["text_model_output"].last_hidden_state
logits.shape, a.shape

(torch.Size([32, 50, 768]), torch.Size([32, 50]))

In [6]:
loss_fn(logits.reshape(-1, 768), a.reshape(-1))

../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# t5 

In [3]:
from transformers import MT5Model, AutoTokenizer, MT5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d")
model = MT5Model.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d")
article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien."
inputs = tokenizer(article, return_tensors="pt")
labels = tokenizer(summary, return_tensors="pt")

outputs = model(input_ids=inputs["input_ids"], decoder_input_ids=labels["input_ids"])
hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at /home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer.decode(inputs.input_ids[0])

'UN Offizier sagt, dass weiter verhandelt werden muss in Syrien.</s>'

In [12]:
model = MT5ForConditionalGeneration.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/snapshots/38f23af8ec210eb6c376d40e9c56bd25a80f195d")


In [2]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer

model = MT5ForConditionalGeneration.from_pretrained("/home/ubuntu/vqa/huggingface_modules/models--google--mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
# article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
# summary = "Weiter Verhandlung in Syrien."
inputs = tokenizer(article, text_target=summary, return_tensors="pt")


OSError: /home/ubuntu/vqa/huggingface_modules/models--google--mt5-small does not appear to have a file named config.json. Checkout 'https://huggingface.co//home/ubuntu/vqa/huggingface_modules/models--google--mt5-small/None' for available files.

In [23]:
inputs1 = tokenizer(article, summary, return_tensors="pt")
inputs2 = tokenizer(article, text_target=summary, return_tensors="pt")

In [25]:
inputs1.input_ids.shape

torch.Size([1, 26])

In [24]:
tokenizer.decode(inputs1.input_ids[0])

'UN Offizier sagt, dass weiter verhandelt werden muss in Syrien.</s> Weiter Verhandlung in Syrien.</s>'

In [23]:
tokenizer.decode(inputs2.input_ids[0])

'UN Offizier sagt, dass weiter verhandelt werden muss in Syrien.</s>'

In [106]:
inputs

{'input_ids': tensor([[ 10969,    443, 209522,    295,    259,   8992,    261,    259,   3648,
           8104,    672,  53764,   2100,   8497,    281,  55984,    278,    260,
              1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [107]:
embed_tokens = model.encoder.embed_tokens(inputs.input_ids)

In [108]:
position_bias = None
for block in model.encoder.block:
    embed_tokens, position_bias = block(embed_tokens, inputs.attention_mask, position_bias=position_bias)
embed_tokens = model.encoder.final_layer_norm(embed_tokens)
embed_tokens = model.encoder.dropout(embed_tokens)

In [109]:
embed_tokens == model.encoder(inputs.input_ids, use_cache=False).last_hidden_state

tensor([[[False, False, False,  ..., False, False, False],
         [False,  True, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [ True, False,  True,  ..., False, False, False]]])

In [110]:
embed_tokens, model.encoder(inputs.input_ids).last_hidden_state

(tensor([[[ 0.1110, -0.0287, -0.4402,  ...,  0.5384,  0.0579,  0.3085],
          [ 0.2239, -0.3732,  0.0509,  ...,  0.3491, -0.2021,  0.0417],
          [ 0.1450, -0.2738, -0.0957,  ...,  0.3628, -0.0667,  0.0604],
          ...,
          [-0.0709, -0.4040, -0.0700,  ..., -0.0504, -0.1645,  0.1855],
          [-0.0890, -0.6575, -0.3589,  ..., -0.0446,  0.0796,  0.0199],
          [-0.0388, -0.0067,  0.0084,  ..., -0.0243,  0.0069,  0.0217]]],
        grad_fn=<MulBackward0>),
 tensor([[[ 0.1110, -0.0287, -0.4402,  ...,  0.5384,  0.0579,  0.3085],
          [ 0.2239, -0.3732,  0.0509,  ...,  0.3491, -0.2021,  0.0417],
          [ 0.1450, -0.2738, -0.0957,  ...,  0.3628, -0.0667,  0.0604],
          ...,
          [-0.0709, -0.4040, -0.0700,  ..., -0.0504, -0.1645,  0.1855],
          [-0.0890, -0.6575, -0.3589,  ..., -0.0446,  0.0796,  0.0199],
          [-0.0388, -0.0067,  0.0084,  ..., -0.0243,  0.0069,  0.0217]]],
        grad_fn=<MulBackward0>))

In [69]:
model.encoder

MT5Stack(
  (embed_tokens): Embedding(250112, 512)
  (block): ModuleList(
    (0): MT5Block(
      (layer): ModuleList(
        (0): MT5LayerSelfAttention(
          (SelfAttention): MT5Attention(
            (q): Linear(in_features=512, out_features=384, bias=False)
            (k): Linear(in_features=512, out_features=384, bias=False)
            (v): Linear(in_features=512, out_features=384, bias=False)
            (o): Linear(in_features=384, out_features=512, bias=False)
            (relative_attention_bias): Embedding(32, 6)
          )
          (layer_norm): MT5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): MT5LayerFF(
          (DenseReluDense): MT5DenseGatedActDense(
            (wi_0): Linear(in_features=512, out_features=1024, bias=False)
            (wi_1): Linear(in_features=512, out_features=1024, bias=False)
            (wo): Linear(in_features=1024, out_features=512, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)

In [71]:
model.encoder.final_layer_norm

MT5LayerNorm()

In [105]:
model.encoder.block[0](embed_tokens)

(tensor([[[-3.3846e+00,  1.4891e+00, -3.0494e-02,  ..., -1.6864e+01,
           -2.2443e+01, -4.4718e+01],
          [-5.8449e+00,  3.8363e+00,  1.9972e-01,  ..., -1.0023e+01,
           -1.7387e+01, -4.0663e+01],
          [-4.4669e+00,  9.1005e+00, -3.1199e+00,  ..., -1.5883e+01,
           -1.9319e+01, -4.0251e+01],
          ...,
          [-1.5249e+00,  3.6194e+00,  2.7254e+00,  ..., -6.7944e+00,
           -2.0742e+01, -4.7024e+01],
          [ 6.6335e-02, -5.5098e+00,  7.5768e-01,  ..., -1.0822e+01,
           -2.2477e+01, -3.8480e+01],
          [-9.4237e+00, -1.7733e+01, -8.1069e+00,  ...,  1.0379e+01,
            4.7321e+01,  2.8246e+01]]], grad_fn=<AddBackward0>),
 tensor([[[[  1.7422,   7.6875,   8.0000,  ...,   6.8438,   6.8438,   6.8438],
           [  7.4375,   1.7422,   7.6875,  ...,   7.1875,   6.8438,   6.8438],
           [  7.6250,   7.4375,   1.7422,  ...,   7.1875,   7.1875,   6.8438],
           ...,
           [  6.4688,   6.7812,   6.7812,  ...,   1.7422,   7.6

In [117]:
outputs.last_hidden_state

tensor([[[-0.4790, -0.0256, -0.3910,  ...,  0.0696,  0.1549,  0.5829],
         [-0.1468,  0.5403, -0.3263,  ...,  0.1398, -0.0957,  0.3944],
         [-0.0460,  0.4281, -0.2796,  ...,  0.1270,  0.0163,  0.4306],
         ...,
         [ 0.0908,  0.2675, -0.2022,  ...,  0.2862, -0.1342,  0.4868],
         [-0.2392,  0.1952, -0.0180,  ...,  0.2424,  0.2441,  0.1907],
         [-0.3405,  0.1611,  0.3945,  ..., -0.0517,  0.0780, -0.1666]]],
       grad_fn=<MulBackward0>)

In [118]:
model.decoder(input_ids=labels.input_ids,
              encoder_hidden_states=embed_tokens).last_hidden_state

tensor([[[-0.4790, -0.0256, -0.3910,  ...,  0.0696,  0.1549,  0.5829],
         [-0.1468,  0.5403, -0.3263,  ...,  0.1398, -0.0957,  0.3944],
         [-0.0460,  0.4281, -0.2796,  ...,  0.1270,  0.0163,  0.4306],
         ...,
         [ 0.0908,  0.2675, -0.2022,  ...,  0.2862, -0.1342,  0.4868],
         [-0.2392,  0.1952, -0.0180,  ...,  0.2424,  0.2441,  0.1907],
         [-0.3405,  0.1611,  0.3945,  ..., -0.0517,  0.0780, -0.1666]]],
       grad_fn=<MulBackward0>)

In [130]:
tokenizer.convert_ids_to_tokens(0)

'<pad>'